In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import ElasticNet, Ridge,LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [11]:
players_df = pd.read_csv('data/players/players_summary.csv')
fixtures_df = pd.read_csv('data/fixture_data.csv')
fdr_df = pd.read_csv('data/fdr.csv')

In [64]:
def combine_data(players, fixtures, fdr):
    fixtures = fixtures.loc[fixtures['game_played'], ['Wk', 'game_id','Home', 'Away']]
    fixtures_home = fixtures.copy().rename(columns={'Home': 'Team'}).merge(
        fdr, how='left', on='Team').drop(columns=['Away'])
    fixtures_home['home'] = True
    fixtures_away = fixtures.copy().rename(columns={'Away': 'Team'}).merge(
        fdr, how='left', on='Team').drop(columns=['Home'])
    fixtures_away['home'] = False
    fixtures_fdr = pd.concat([fixtures_home, fixtures_away], ignore_index=True)
    players_fdr = players.merge(fixtures_fdr, how='left', on=['game_id', 'home'])
    return players_fdr

def generate_ts_data(df, N):
    top_players = df.groupby('Player', as_index=False).sum()
    top_players = top_players[['Player', 'xG']]
    top_players = top_players.sort_values('xG', ascending=False)
    top_players = top_players.head(N)['Player'].to_list()
    ts_data = df.loc[df['Player'].isin(top_players), ['Player', 'Wk', 'xG']]
    
    all_weeks = range(df['Wk'].min(), df['Wk'].max() + 1)
    all_players = ts_data['Player'].unique()
    full_index = pd.MultiIndex.from_product([all_players, all_weeks], names=['Player', 'Wk'])
    ts_data = ts_data.set_index(['Player', 'Wk']).reindex(full_index, fill_value=0).reset_index()
    return ts_data

In [65]:
N=30
players_fdr = combine_data(players_df, fixtures_df, fdr_df)
ts_data = generate_ts_data(players_fdr, N)

ts_data.sample(5)

,Player,Wk,xG
24,Antoine Semenyo,1,0.9
6,Hugo Ekitike,7,0.1
93,Thiago,6,0.7
109,Estêvão Willian,6,0.2
104,Estêvão Willian,1,0.3


In [68]:
np.random.seed(42)
groups = ts_data['Player'].nunique()
periods = ts_data['Wk'].nunique()
df = ts_data.rename(columns={'Player': 'group', 'Wk': 'time', 'xG': 'variable'})

## Statistical Methods

In [69]:
print("Data shape:", df.shape)
print("Groups:", df['group'].nunique())
print("Periods:", df['time'].nunique())

# ============================================================================
# APPROACH 1: STACKING - Use all data points
# ============================================================================

print("\n" + "="*70)
print("APPROACH 1: STACKING ALL OBSERVATIONS")
print("="*70)

# Create lagged features for each group separately
df_features = []
for g in df['group'].unique():
    group_df = df[df['group'] == g].copy().sort_values('time')
    group_df['lag1'] = group_df['variable'].shift(1)
    group_df['lag2'] = group_df['variable'].shift(2)
    group_df['time_linear'] = group_df['time']
    df_features.append(group_df)

df_stacked = pd.concat(df_features, ignore_index=True)
df_stacked = df_stacked.dropna()  # Remove rows with NaN from lagging

print(f"Stacked data shape: {df_stacked.shape}")
print(f"Using {len(df_stacked)} observations for modeling")

# Simple AR model using all data
X = df_stacked[['lag1', 'time_linear']]
y = df_stacked['variable']

# Fit AR model
ar_model = LinearRegression()
ar_model.fit(X, y)

predictions = ar_model.predict(X)
stacked_mse = mean_squared_error(y, predictions)
stacked_rmse = np.sqrt(stacked_mse)

print(f"\nAR(1) + Trend Model:")
print(f"  Coefficient on lag1: {ar_model.coef_[0]:.4f}")
print(f"  Coefficient on time: {ar_model.coef_[1]:.4f}")
print(f"  Intercept: {ar_model.intercept_:.4f}")
print(f"  RMSE: {stacked_rmse:.4f}")

# ============================================================================
# APPROACH 2: STACKED ARIMA
# ============================================================================

print("\n" + "="*70)
print("APPROACH 2: FIT ARIMA ON EACH GROUP, AVERAGE PARAMETERS")
print("="*70)

# Fit ARIMA on each group and collect parameters
arima_params = []
group_mse = []

for g in df['group'].unique():
    group_data = df[df['group'] == g].sort_values('time')['variable'].values
    try:
        model = ARIMA(group_data, order=(1, 0, 0))
        fit = model.fit()
        arima_params.append({
            'group': g,
            'ar_coef': fit.params[1],
            'intercept': fit.params[0],
            'sigma': np.sqrt(fit.params[2])
        })
        fitted = fit.fittedvalues
        mse = mean_squared_error(group_data[1:], fitted[1:])
        group_mse.append(mse)
    except:
        pass

params_df = pd.DataFrame(arima_params)
print(f"\nFitted ARIMA(1,0,0) on {len(params_df)} groups")
print("\nParameter estimates (mean ± std):")
print(f"  AR coefficient: {params_df['ar_coef'].mean():.4f} ± {params_df['ar_coef'].std():.4f}")
print(f"  Intercept: {params_df['intercept'].mean():.4f} ± {params_df['intercept'].std():.4f}")
print(f"  Mean group MSE: {np.mean(group_mse):.4f}")

# Use average parameters for new groups
avg_ar = params_df['ar_coef'].mean()
avg_intercept = params_df['intercept'].mean()

print(f"\nPooled ARIMA parameters to use for new groups:")
print(f"  AR(1) coefficient: {avg_ar:.4f}")
print(f"  Intercept: {avg_intercept:.4f}")

# ============================================================================
# APPROACH 3: MIXED EFFECTS MODEL (Group Random Effects)
# ============================================================================

print("\n" + "="*70)
print("APPROACH 3: MIXED EFFECTS WITH GROUP RANDOM EFFECTS")
print("="*70)

try:
    # Create lagged variable
    df_mixed = []
    for g in df['group'].unique():
        group_df = df[df['group'] == g].copy().sort_values('time')
        group_df['lag1'] = group_df['variable'].shift(1)
        df_mixed.append(group_df)
    
    df_mixed = pd.concat(df_mixed, ignore_index=True).dropna()
    
    # Mixed effects model: random intercept for each group
    mixed_model = smf.mixedlm("variable ~ lag1 + time", df_mixed, groups=df_mixed["group"])
    mixed_results = mixed_model.fit()
    
    print(mixed_results.summary())
    
    # Extract fixed effects
    fixed_effects = mixed_results.params
    print(f"\nFixed effects:")
    print(f"  Lag1 coefficient: {fixed_effects['lag1']:.4f}")
    print(f"  Time coefficient: {fixed_effects['time']:.4f}")
    print(f"  Intercept: {fixed_effects['Intercept']:.4f}")
    
except Exception as e:
    print(f"Mixed effects model failed: {e}")

# ============================================================================
# APPROACH 4: TIME SERIES CROSS-VALIDATION
# ============================================================================

print("\n" + "="*70)
print("APPROACH 4: TIME SERIES CROSS-VALIDATION")
print("="*70)

# Use first 6 periods for training, last 2 for testing
train_periods = 6
test_periods = 2

train_df = df[df['time'] <= train_periods].copy()
test_df = df[df['time'] > train_periods].copy()

# Create lagged features for training
train_features = []
for g in train_df['group'].unique():
    group_df = train_df[train_df['group'] == g].copy().sort_values('time')
    group_df['lag1'] = group_df['variable'].shift(1)
    train_features.append(group_df)

train_stacked = pd.concat(train_features, ignore_index=True).dropna()

# Fit model on training data
X_train = train_stacked[['lag1', 'time']]
y_train = train_stacked['variable']

cv_model = LinearRegression()
cv_model.fit(X_train, y_train)

# Test on held-out periods
test_predictions = []
test_actuals = []

for g in test_df['group'].unique():
    group_test = test_df[test_df['group'] == g].sort_values('time')
    group_train = train_df[train_df['group'] == g].sort_values('time')
    
    # Last training value as lag
    last_train_val = group_train['variable'].values[-1]
    
    for idx, row in group_test.iterrows():
        pred = cv_model.predict([[last_train_val, row['time']]])[0]
        test_predictions.append(pred)
        test_actuals.append(row['variable'])
        last_train_val = row['variable']  # Use actual for next prediction

test_mse = mean_squared_error(test_actuals, test_predictions)
test_rmse = np.sqrt(test_mse)

print(f"Training periods: 1-{train_periods}")
print(f"Test periods: {train_periods+1}-{periods}")
print(f"Test RMSE: {test_rmse:.4f}")

# ============================================================================
# SUMMARY AND RECOMMENDATIONS
# ============================================================================

print("\n" + "="*70)
print("SUMMARY: MODEL COMPARISON")
print("="*70)

summary = pd.DataFrame({
    'Approach': [
        'Stacking (AR+trend)',
        'Group ARIMA average',
        'Time Series CV'
    ],
    'RMSE': [
        stacked_rmse,
        np.sqrt(np.mean(group_mse)),
        test_rmse
    ],
    'Observations': [
        len(df_stacked),
        f"{len(params_df)} groups",
        f"{len(test_actuals)} test obs"
    ]
})

print(summary.to_string(index=False))

print("\n" + "="*70)
print("APPLYING TO NEW GROUP")
print("="*70)

print("""
Method 1 (Stacking): Use fitted AR model directly
    new_X = pd.DataFrame({'lag1': [prev_value], 'time_linear': [time_period]})
    prediction = ar_model.predict(new_X)

Method 2 (Pooled ARIMA): Use average AR coefficient
    prediction = avg_intercept + avg_ar * prev_value
    
Method 3 (Fit new ARIMA): Fit ARIMA(1,0,0) on new group's 8 periods
    model = ARIMA(new_group_data, order=(1,0,0))
    fit = model.fit()
    forecast = fit.forecast(steps=h)
""")

Data shape: (240, 3)
Groups: 30
Periods: 8

APPROACH 1: STACKING ALL OBSERVATIONS
Stacked data shape: (180, 6)
Using 180 observations for modeling

AR(1) + Trend Model:
  Coefficient on lag1: 0.1349
  Coefficient on time: 0.0137
  Intercept: 0.2266
  RMSE: 0.4427

APPROACH 2: FIT ARIMA ON EACH GROUP, AVERAGE PARAMETERS

Fitted ARIMA(1,0,0) on 30 groups

Parameter estimates (mean ± std):
  AR coefficient: -0.1010 ± 0.3748
  Intercept: 0.3640 ± 0.2036
  Mean group MSE: 0.1359

Pooled ARIMA parameters to use for new groups:
  AR(1) coefficient: -0.1010
  Intercept: 0.3640

APPROACH 3: MIXED EFFECTS WITH GROUP RANDOM EFFECTS
         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: variable 
No. Observations: 210     Method:             REML     
No. Groups:       30      Scale:              0.1823   
Min. group size:  7       Log-Likelihood:     -131.4561
Max. group size:  7       Converged:          No       
Mean group size:  7.0                       

## ML Approaches

In [70]:
# ============================================================================
# FEATURE ENGINEERING
# ============================================================================

def create_features(df, max_lag=3):
    """Create lagged features and other time series features"""
    df_features = []
    
    for g in df['group'].unique():
        group_df = df[df['group'] == g].copy().sort_values('time')
        
        # Lagged values
        for i in range(1, max_lag + 1):
            group_df[f'lag{i}'] = group_df['variable'].shift(i)
        
        # Rolling statistics
        group_df['roll_mean_3'] = group_df['variable'].rolling(window=3).mean()
        group_df['roll_std_3'] = group_df['variable'].rolling(window=3).std()
        
        # Time features
        group_df['time'] = group_df['time']
        group_df['time_squared'] = group_df['time'] ** 2
        
        # Differences
        group_df['diff1'] = group_df['variable'].diff(1)
        group_df['diff2'] = group_df['variable'].diff(2)
        
        df_features.append(group_df)
    
    result = pd.concat(df_features, ignore_index=True)
    return result.dropna()

df_ml = create_features(df, max_lag=2)

print("Feature engineering complete")
print(f"Original data: {len(df)} rows")
print(f"After feature engineering: {len(df_ml)} rows")
print(f"\nFeatures: {[col for col in df_ml.columns if col not in ['group', 'variable']]}")

# Prepare data
feature_cols = [col for col in df_ml.columns if col not in ['group', 'variable']]
X = df_ml[feature_cols]
y = df_ml['variable']

# Split train/test by time
train_mask = df_ml['time'] <= 6
X_train, X_test = X[train_mask], X[~train_mask]
y_train, y_test = y[train_mask], y[~train_mask]

print(f"\nTrain size: {len(X_train)}, Test size: {len(X_test)}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ============================================================================
# MODEL 1: RANDOM FOREST
# ============================================================================

print("\n" + "="*70)
print("MODEL 1: RANDOM FOREST")
print("="*70)

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=5,
    min_samples_split=5,
    random_state=42
)

rf_model.fit(X_train, y_train)
rf_pred_train = rf_model.predict(X_train)
rf_pred_test = rf_model.predict(X_test)

rf_train_rmse = np.sqrt(mean_squared_error(y_train, rf_pred_train))
rf_test_rmse = np.sqrt(mean_squared_error(y_test, rf_pred_test))
rf_test_r2 = r2_score(y_test, rf_pred_test)

print(f"Train RMSE: {rf_train_rmse:.4f}")
print(f"Test RMSE: {rf_test_rmse:.4f}")
print(f"Test R²: {rf_test_r2:.4f}")

# Feature importance
feature_imp = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 5 features:")
print(feature_imp.head().to_string(index=False))

# ============================================================================
# MODEL 2: GRADIENT BOOSTING
# ============================================================================

print("\n" + "="*70)
print("MODEL 2: GRADIENT BOOSTING")
print("="*70)

gb_model = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42
)

gb_model.fit(X_train, y_train)
gb_pred_train = gb_model.predict(X_train)
gb_pred_test = gb_model.predict(X_test)

gb_train_rmse = np.sqrt(mean_squared_error(y_train, gb_pred_train))
gb_test_rmse = np.sqrt(mean_squared_error(y_test, gb_pred_test))
gb_test_r2 = r2_score(y_test, gb_pred_test)

print(f"Train RMSE: {gb_train_rmse:.4f}")
print(f"Test RMSE: {gb_test_rmse:.4f}")
print(f"Test R²: {gb_test_r2:.4f}")

# ============================================================================
# MODEL 3: RIDGE REGRESSION (Regularized Linear)
# ============================================================================

print("\n" + "="*70)
print("MODEL 3: RIDGE REGRESSION")
print("="*70)

ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train_scaled, y_train)
ridge_pred_train = ridge_model.predict(X_train_scaled)
ridge_pred_test = ridge_model.predict(X_test_scaled)

ridge_train_rmse = np.sqrt(mean_squared_error(y_train, ridge_pred_train))
ridge_test_rmse = np.sqrt(mean_squared_error(y_test, ridge_pred_test))
ridge_test_r2 = r2_score(y_test, ridge_pred_test)

print(f"Train RMSE: {ridge_train_rmse:.4f}")
print(f"Test RMSE: {ridge_test_rmse:.4f}")
print(f"Test R²: {ridge_test_r2:.4f}")

# Top coefficients
coef_df = pd.DataFrame({
    'feature': feature_cols,
    'coefficient': ridge_model.coef_
}).sort_values('coefficient', key=abs, ascending=False)

print("\nTop 5 coefficients:")
print(coef_df.head().to_string(index=False))

# ============================================================================
# MODEL 4: ELASTIC NET (L1 + L2 Regularization)
# ============================================================================

print("\n" + "="*70)
print("MODEL 4: ELASTIC NET")
print("="*70)

elastic_model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
elastic_model.fit(X_train_scaled, y_train)
elastic_pred_train = elastic_model.predict(X_train_scaled)
elastic_pred_test = elastic_model.predict(X_test_scaled)

elastic_train_rmse = np.sqrt(mean_squared_error(y_train, elastic_pred_train))
elastic_test_rmse = np.sqrt(mean_squared_error(y_test, elastic_pred_test))
elastic_test_r2 = r2_score(y_test, elastic_pred_test)

print(f"Train RMSE: {elastic_train_rmse:.4f}")
print(f"Test RMSE: {elastic_test_rmse:.4f}")
print(f"Test R²: {elastic_test_r2:.4f}")

# Check sparsity
non_zero_coefs = np.sum(elastic_model.coef_ != 0)
print(f"Non-zero coefficients: {non_zero_coefs}/{len(feature_cols)}")

# ============================================================================
# MODEL 5: NEURAL NETWORK (MLP)
# ============================================================================

print("\n" + "="*70)
print("MODEL 5: NEURAL NETWORK (MLP)")
print("="*70)

mlp_model = MLPRegressor(
    hidden_layer_sizes=(32, 16),
    activation='relu',
    max_iter=1000,
    random_state=42,
    early_stopping=True
)

mlp_model.fit(X_train_scaled, y_train)
mlp_pred_train = mlp_model.predict(X_train_scaled)
mlp_pred_test = mlp_model.predict(X_test_scaled)

mlp_train_rmse = np.sqrt(mean_squared_error(y_train, mlp_pred_train))
mlp_test_rmse = np.sqrt(mean_squared_error(y_test, mlp_pred_test))
mlp_test_r2 = r2_score(y_test, mlp_pred_test)

print(f"Train RMSE: {mlp_train_rmse:.4f}")
print(f"Test RMSE: {mlp_test_rmse:.4f}")
print(f"Test R²: {mlp_test_r2:.4f}")

# ============================================================================
# MODEL COMPARISON
# ============================================================================

print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)

comparison = pd.DataFrame({
    'Model': ['Random Forest', 'Gradient Boosting', 'Ridge', 'Elastic Net', 'Neural Network'],
    'Train_RMSE': [rf_train_rmse, gb_train_rmse, ridge_train_rmse, elastic_train_rmse, mlp_train_rmse],
    'Test_RMSE': [rf_test_rmse, gb_test_rmse, ridge_test_rmse, elastic_test_rmse, mlp_test_rmse],
    'Test_R2': [rf_test_r2, gb_test_r2, ridge_test_r2, elastic_test_r2, mlp_test_r2]
})

comparison['Overfit'] = comparison['Train_RMSE'] - comparison['Test_RMSE']
comparison = comparison.sort_values('Test_RMSE')

print(comparison.to_string(index=False))

# ============================================================================
# ENSEMBLE: WEIGHTED AVERAGE
# ============================================================================

print("\n" + "="*70)
print("ENSEMBLE: SIMPLE AVERAGE")
print("="*70)

ensemble_pred_test = (rf_pred_test + gb_pred_test + ridge_pred_test) / 3
ensemble_rmse = np.sqrt(mean_squared_error(y_test, ensemble_pred_test))
ensemble_r2 = r2_score(y_test, ensemble_pred_test)

print(f"Ensemble Test RMSE: {ensemble_rmse:.4f}")
print(f"Ensemble Test R²: {ensemble_r2:.4f}")

# ============================================================================
# VISUALIZE PREDICTIONS
# ============================================================================

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

models = [
    ('Random Forest', rf_pred_test),
    ('Gradient Boosting', gb_pred_test),
    ('Ridge', ridge_pred_test),
    ('Elastic Net', elastic_pred_test),
    ('Neural Network', mlp_pred_test),
    ('Ensemble', ensemble_pred_test)
]

for idx, (name, preds) in enumerate(models):
    ax = axes[idx]
    ax.scatter(y_test, preds, alpha=0.6)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    ax.set_xlabel('Actual')
    ax.set_ylabel('Predicted')
    ax.set_title(name)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('ml_predictions.png', dpi=300)
plt.close()

print("\nPrediction plots saved as 'ml_predictions.png'")

# ============================================================================
# APPLY TO NEW GROUP
# ============================================================================

print("\n" + "="*70)
print("APPLYING TO NEW GROUP")
print("="*70)

def predict_new_group(new_group_df, model, scaler=None, scale=False):
    """
    new_group_df: DataFrame with 'time' and 'variable' columns
    model: trained sklearn model
    scaler: fitted StandardScaler (if scaling needed)
    scale: whether to scale features
    """
    new_features = create_features(
        new_group_df.assign(group=999), 
        max_lag=2
    )
    
    X_new = new_features[feature_cols]
    
    if scale and scaler:
        X_new = scaler.transform(X_new)
    
    predictions = model.predict(X_new)
    
    new_features['prediction'] = predictions
    return new_features[['time', 'variable', 'prediction']]

print("""
Example usage:

# Load new group data
new_group = pd.DataFrame({
    'time': [1, 2, 3, 4, 5, 6, 7, 8],
    'variable': [10.2, 11.1, 10.8, 12.3, 13.1, 12.9, 14.0, 14.5]
})

# Make predictions
predictions = predict_new_group(new_group, rf_model, scale=False)

# For models needing scaling (Ridge, Elastic Net, MLP)
predictions = predict_new_group(new_group, ridge_model, scaler, scale=True)
""")

# ============================================================================
# RECOMMENDATIONS
# ============================================================================

print("\n" + "="*70)
print("RECOMMENDATIONS")
print("="*70)

print("""
Best ML Approaches for Your Case:

1. GRADIENT BOOSTING (Recommended)
   ✓ Handles non-linearity well
   ✓ Good with limited data
   ✓ Less prone to overfitting than Random Forest
   ✓ Built-in feature importance

2. RANDOM FOREST
   ✓ Robust to outliers
   ✓ Handles feature interactions
   ✗ Can overfit with small data
   
3. RIDGE/ELASTIC NET
   ✓ Interpretable coefficients
   ✓ Regularization prevents overfitting
   ✓ Works well as baseline
   ✗ Assumes linear relationships

4. ENSEMBLE
   ✓ Often best performance
   ✓ Reduces model-specific errors
   
Key Feature Engineering Tips:
- Lag features (1-3 lags) are most important
- Rolling statistics capture recent trends
- Time and time² capture non-linear trends
- Differences capture changes

With 8 periods per group:
- Keep models simple (low max_depth, regularization)
- Feature engineering matters more than model complexity
- Cross-validation is critical
""")

Feature engineering complete
Original data: 240 rows
After feature engineering: 180 rows

Features: ['time', 'lag1', 'lag2', 'roll_mean_3', 'roll_std_3', 'time_squared', 'diff1', 'diff2']

Train size: 120, Test size: 60

MODEL 1: RANDOM FOREST
Train RMSE: 0.0744
Test RMSE: 0.3015
Test R²: 0.6957

Top 5 features:
    feature  importance
      diff1    0.492027
      diff2    0.303916
 roll_std_3    0.094969
roll_mean_3    0.081734
       lag1    0.019239

MODEL 2: GRADIENT BOOSTING
Train RMSE: 0.0149
Test RMSE: 0.2915
Test R²: 0.7156

MODEL 3: RIDGE REGRESSION
Train RMSE: 0.0030
Test RMSE: 0.0033
Test R²: 1.0000

Top 5 coefficients:
    feature  coefficient
roll_mean_3     0.200210
      diff1     0.197744
      diff2     0.189807
       lag1     0.047978
       lag2     0.033293

MODEL 4: ELASTIC NET
Train RMSE: 0.0917
Test RMSE: 0.1297
Test R²: 0.9437
Non-zero coefficients: 3/8

MODEL 5: NEURAL NETWORK (MLP)
Train RMSE: 0.0265
Test RMSE: 0.1134
Test R²: 0.9569

MODEL COMPARISON
      